# Exploratory Data Analysis - Competitor Pricing Optimizer

This notebook performs comprehensive EDA on the product data, including:
- Data loading and inspection
- Missing value analysis
- Feature engineering
- Statistical analysis
- Correlation analysis
- Initial clustering exploration


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)


## 1. Load Data


In [ ]:
# Load raw data
df_raw = pd.read_csv('../data/raw/products.csv')
print(f"Raw data shape: {df_raw.shape}")
df_raw.head()


In [ ]:
# Load processed data (if available)
try:
    df_processed = pd.read_csv('../data/processed/products_processed.csv')
    print(f"Processed data shape: {df_processed.shape}")
    df_processed.head()
except FileNotFoundError:
    print("Processed data not found. Run preprocessing first.")
    df_processed = None


## 2. Data Overview


In [ ]:
df = df_raw.copy()

print("Data Info:")
print(df.info())
print("\n" + "="*50)
print("\nData Description:")
print(df.describe())


## 3. Missing Values Analysis


In [ ]:
missing_data = df.isnull().sum()
missing_pct = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing %': missing_pct
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print("Missing Values:")
    print(missing_df)
    
    # Visualize
    fig = px.bar(
        missing_df.reset_index(),
        x='index',
        y='Missing %',
        title='Missing Values by Column',
        labels={'index': 'Column', 'Missing %': 'Missing Percentage'}
    )
    fig.update_layout(height=400)
    fig.show()
else:
    print("No missing values found!")


## 4. Feature Distribution Analysis


In [ ]:
# Price distribution
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Price Distribution', 'Rating Distribution', 'Reviews Distribution', 'Price vs Rating'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Price histogram
fig.add_trace(
    go.Histogram(x=df['price'], nbinsx=50, name='Price'),
    row=1, col=1
)

# Rating histogram
fig.add_trace(
    go.Histogram(x=df['rating'], nbinsx=20, name='Rating'),
    row=1, col=2
)

# Reviews histogram
fig.add_trace(
    go.Histogram(x=df['reviews_count'], nbinsx=50, name='Reviews'),
    row=2, col=1
)

# Scatter plot
fig.add_trace(
    go.Scatter(x=df['price'], y=df['rating'], mode='markers', name='Price vs Rating'),
    row=2, col=2
)

fig.update_layout(height=800, showlegend=False, title_text="Feature Distributions")
fig.update_xaxes(title_text="Price (₹)", row=1, col=1)
fig.update_xaxes(title_text="Rating", row=1, col=2)
fig.update_xaxes(title_text="Reviews Count", row=2, col=1)
fig.update_xaxes(title_text="Price (₹)", row=2, col=2)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_yaxes(title_text="Count", row=2, col=1)
fig.update_yaxes(title_text="Rating", row=2, col=2)

fig.show()


In [ ]:
if 'brand' in df.columns:
    brand_stats = df.groupby('brand').agg({
        'price': ['mean', 'std', 'count'],
        'rating': 'mean',
        'reviews_count': 'mean'
    }).round(2)
    
    brand_stats.columns = ['Avg Price', 'Price Std', 'Product Count', 'Avg Rating', 'Avg Reviews']
    brand_stats = brand_stats.sort_values('Avg Price', ascending=False)
    
    print("Brand Statistics:")
    print(brand_stats)
    
    # Visualize
    fig = px.bar(
        brand_stats.reset_index(),
        x='brand',
        y='Avg Price',
        title='Average Price by Brand',
        labels={'brand': 'Brand', 'Avg Price': 'Average Price (₹)'}
    )
    fig.update_layout(height=400)
    fig.show()


## 6. Correlation Analysis


In [ ]:
# Select numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
corr_matrix = df[numeric_cols].corr()

# Heatmap
fig = px.imshow(
    corr_matrix,
    labels=dict(color="Correlation"),
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    color_continuous_scale='RdBu',
    title='Correlation Heatmap'
)
fig.update_layout(height=600)
fig.show()

print("\nCorrelation Matrix:")
print(corr_matrix.round(3))


## 7. Feature Engineering Preview


In [ ]:
# Calculate derived features
df['discount_pct'] = ((df['original_price'] - df['price']) / df['original_price'] * 100).fillna(0)
df['demand_proxy'] = df['rating'] * df['reviews_count']
df['price_per_rating'] = df['price'] / (df['rating'] + 0.1)

print("Engineered Features:")
print(df[['discount_pct', 'demand_proxy', 'price_per_rating']].describe())


## 8. Initial Clustering Exploration


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# Prepare features for clustering
features = ['price', 'rating', 'reviews_count', 'demand_proxy']
X = df[features].fillna(df[features].median())

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Test different numbers of clusters
silhouette_scores = []
inertias = []
k_range = range(2, 10)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    silhouette_scores.append(silhouette_score(X_scaled, labels))
    inertias.append(kmeans.inertia_)

# Plot elbow curve
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Elbow Method', 'Silhouette Score'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}]]
)

fig.add_trace(
    go.Scatter(x=list(k_range), y=inertias, mode='lines+markers', name='Inertia'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=list(k_range), y=silhouette_scores, mode='lines+markers', name='Silhouette Score'),
    row=1, col=2
)

fig.update_layout(height=400, title_text="Clustering Analysis")
fig.update_xaxes(title_text="Number of Clusters", row=1, col=1)
fig.update_xaxes(title_text="Number of Clusters", row=1, col=2)
fig.update_yaxes(title_text="Inertia", row=1, col=1)
fig.update_yaxes(title_text="Silhouette Score", row=1, col=2)

fig.show()

optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"\nOptimal number of clusters: {optimal_k} (Silhouette Score: {max(silhouette_scores):.3f})")


In [ ]:
print("="*60)
print("KEY INSIGHTS")
print("="*60)
print(f"\n1. Total Products: {len(df)}")
print(f"2. Price Range: ₹{df['price'].min():,.0f} - ₹{df['price'].max():,.0f}")
print(f"3. Average Price: ₹{df['price'].mean():,.0f}")
print(f"4. Average Rating: {df['rating'].mean():.2f}")
print(f"5. Total Reviews: {df['reviews_count'].sum():,}")

if 'brand' in df.columns:
    top_brand = df.groupby('brand')['price'].mean().idxmax()
    print(f"6. Highest Priced Brand: {top_brand}")

print(f"\n7. Optimal Clusters: {optimal_k}")
print("="*60)
